# Earth / Moon vs. Earth / Mars Barcodes

### Setup Base

In [2]:
from os_utilities import *
from orb_builder import *
import time_utilities as tu

orb_filenames = [] # used to keep track of what files to feed into soap


prefix = "starlink" # earth / moon / mars
n = 1 # number of simulations to generate
k = 50 # number of satellites to sample from starlink

folder_name = "sim-2023-03-23-rnd"
# folder_name = "starlink-{}-sat-single".format(k)

folder = "./outputs/" + folder_name
make_folder(folder)

starlink = build_satellites_csv("starlink")
moon_sats = get_moon_platforms()
mars_sats = get_mars_platforms()
ground = [
    create_ground_object_dictionary("Ground-Albany", 42.685012663456163, -73.82479012295363, 0),
    create_ground_object_dictionary("Ground-Sydney", -33.868888888888889, 151.20939697339508, 0)
]
norad = build_satellites_from_tle("./sources/tdrss.tle")

import datetime
import random

year = 2022
months = 24 # 8 years
months = 12 # 8 years
month_days = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

dates = []
for j in range(months):
    
    if j > 0 and j % 12 == 0:
        year += 1
    month_index = j % 12
    day = random.randint(1, month_days[month_index])
    # day = 1
    date = datetime.datetime(year, month_index + 1, day)
    date = tu.get_random_date()
    dates.append(date)
# print(len(dates))
# print(dates)
# dates = []


In [3]:
for i in range(n):
    for d in dates:
        name = prefix + "_" + str(k) + "_sats_" + d.strftime("%Y-%m") + "_" + str(i) # "%Y-%m-%d"

        subset = sample_sats(starlink, k)

        # track which satellites were chosen for subsequent simulations
        subset_names = []
        for satellite in subset:
            subset_names.append(satellite["object_name"])

        satellites = subset # + moon_sats # + ground

        orb_filenames.append(name + ".orb")

        save_to_outputs_file(
            generate_orb(satellites, name, d),
            folder_name + "/" + name, "orb")

## Earth / Moon System

In [2]:
for i in range(n):
    for d in dates:

        name = prefix + "_moon_" + d.strftime("%Y-%m") + "_" + str(i)

        subset = sample_sats(starlink, k)

        # track which satellites were chosen for subsequent simulations
        subset_names = []
        for satellite in subset:
            subset_names.append(satellite["object_name"])

        satellites = subset + moon_sats # + ground

        orb_filenames.append(name + ".orb")

        save_to_outputs_file(
            generate_orb(satellites, name, d),
            folder_name + "/" + name, "orb")

## Earth / Mars System

In [3]:
for i in range(n):
    for d in dates:

        name = prefix + "_mars_" + d.strftime("%Y-%m") + "_" + str(i)

        subset = sample_sats(starlink, k + 1)

        # track which satellites were chosen for subsequent simulations
        subset_names = []
        for satellite in subset:
            subset_names.append(satellite["object_name"])

        satellites = subset + mars_sats # + ground

        orb_filenames.append(name + ".orb")

        save_to_outputs_file(
            generate_orb(satellites, name, d),
            folder_name + "/" + name, "orb")

## Run Soap

In [3]:
# folder = "outputs/starlink-6"
# orb_files = get_ext_files(folder, "orb")
# orb_filenames = []
# for orb_file in orb_files:
#     orb_filenames.append(orb_file.split("/")[-1])
# print(orb_filenames)
bash = starlink_bash = "#!/bin/bash" + "\n"

for filename in orb_filenames:
    bash += f"open -j -W {filename}\n"
save_to_outputs_file(
        bash,
        folder_name + "/run",
        "sh")

0

In [7]:
import subprocess

command = "wine"
soap_path = "/home/zeno/.wine/drive_c/SOAP/bin64/soap.exe"

command = "open"
soap_path = ""

for filename in orb_filenames:
    # args = ["open", "-n", "-W", f"{folder}/{filename}"]
    args = ["open", "-n", "-j", "-W", f"{folder}/{filename}"]
    print(" ".join(args))
    # orb_path = "" + folder + "/" + filename + ""
    # process = subprocess.Popen(args)
    # process = subprocess.Popen([command, orb_path], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
    process = subprocess.Popen(args)
    process.wait()

open -W ./outputs/sim-2023-03-23-rnd/starlink_50_sats_2022-08_0.orb --args -nogui
open -W ./outputs/sim-2023-03-23-rnd/starlink_50_sats_1974-08_0.orb --args -nogui
open -W ./outputs/sim-2023-03-23-rnd/starlink_50_sats_2012-05_0.orb --args -nogui
open -W ./outputs/sim-2023-03-23-rnd/starlink_50_sats_1955-10_0.orb --args -nogui
open -W ./outputs/sim-2023-03-23-rnd/starlink_50_sats_1900-09_0.orb --args -nogui
open -W ./outputs/sim-2023-03-23-rnd/starlink_50_sats_1863-11_0.orb --args -nogui
open -W ./outputs/sim-2023-03-23-rnd/starlink_50_sats_1809-05_0.orb --args -nogui
open -W ./outputs/sim-2023-03-23-rnd/starlink_50_sats_1969-07_0.orb --args -nogui
open -W ./outputs/sim-2023-03-23-rnd/starlink_50_sats_1930-10_0.orb --args -nogui
open -W ./outputs/sim-2023-03-23-rnd/starlink_50_sats_1918-06_0.orb --args -nogui
open -W ./outputs/sim-2023-03-23-rnd/starlink_50_sats_1879-09_0.orb --args -nogui
open -W ./outputs/sim-2023-03-23-rnd/starlink_50_sats_1837-10_0.orb --args -nogui


## Generate Barcodes

In [ ]:
from zz_persistence import *

diagrams = {}
diagrams_clique = {}

filepaths = get_csv_files(folder)
for filepath in filepaths:
    if not filepath.endswith("Contact Analysis.csv"):
        continue
    contact_plan = ca.contact_analysis_parser(filepath)
    graph = ca.construct_graph(contact_plan)
    
    weighted_simplex = ca.construct_weighted_simplex(graph)
    clique_complex = ca.construct_clique_complex(graph)
    
    zz, dgms, cells = calculate_zz_persistence(weighted_simplex)
    zz_clique, dgms_clique, cells_clique = calculate_zz_persistence(clique_complex)
    
    diagrams[filepath.split("/")[-1]] = dgms
    diagrams_clique[filepath.split("/")[-1]] = dgms_clique
    
# print(diagrams)
# print(diagrams_clique)

In [ ]:
dim = 1

## Compare Earth / Moon Simulations

In [ ]:
# Compare Earth / Moon
for i in range(n):
    for j in range(i + 1, n):
        print("{} : {}".format(i, j))

        diagram_a = diagrams["emm_moon_" + str(i) + " Contact Analysis.csv"]
        diagram_b = diagrams["emm_moon_" + str(j) + " Contact Analysis.csv"]
        
        diagram_a_clique = diagrams_clique["emm_moon_" + str(i) + " Contact Analysis.csv"]
        diagram_b_clique = diagrams_clique["emm_moon_" + str(j) + " Contact Analysis.csv"]

        wdist = d.wasserstein_distance(diagram_a[dim], diagram_b[dim], q=2)
        print("2-Wasserstein distance between {}-dimensional persistence diagrams: {}".format(dim, wdist))

        bdist = d.bottleneck_distance(diagram_a[dim], diagram_b[dim])
        print("Bottleneck distance between {}-dimensional persistence diagrams: {}".format(dim, bdist))
        
        wdist_clique = d.wasserstein_distance(diagram_a_clique[dim], diagram_b_clique[dim], q=2)
        print("2-Wasserstein distance between {}-dimensional persistence diagrams (Clique): {}".format(dim, wdist_clique))

        bdist_clique = d.bottleneck_distance(diagram_a_clique[dim], diagram_b_clique[dim])
        print("Bottleneck distance between {}-dimensional persistence diagrams (Clique): {}".format(dim, bdist_clique))
        
        

## Compare Earth / Mars Simulations

In [ ]:
# Compare Earth / Mars
for i in range(n):
    for j in range(i + 1, n):
        print("{} : {}".format(i, j))

        diagram_a = diagrams["emm_mars_" + str(i) + " Contact Analysis.csv"]
        diagram_b = diagrams["emm_mars_" + str(j) + " Contact Analysis.csv"]

        diagram_a_clique = diagrams_clique["emm_mars_" + str(i) + " Contact Analysis.csv"]
        diagram_b_clique = diagrams_clique["emm_mars_" + str(j) + " Contact Analysis.csv"]

        wdist = d.wasserstein_distance(diagram_a[dim], diagram_b[dim], q=2)
        print("2-Wasserstein distance between {}-dimensional persistence diagrams: {}".format(dim, wdist))

        bdist = d.bottleneck_distance(diagram_a[dim], diagram_b[dim])
        print("Bottleneck distance between {}-dimensional persistence diagrams: {}".format(dim, bdist))
        
        wdist_clique = d.wasserstein_distance(diagram_a_clique[dim], diagram_b_clique[dim], q=2)
        print("2-Wasserstein distance between {}-dimensional persistence diagrams (Clique): {}".format(dim, wdist_clique))

        bdist_clique = d.bottleneck_distance(diagram_a_clique[dim], diagram_b_clique[dim])
        print("Bottleneck distance between {}-dimensional persistence diagrams (Clique): {}".format(dim, bdist_clique))
        
        

## Cross Compare Moon / Mars

In [ ]:
# Cross Compare Moon / Mars
for i in range(n):
    for j in range(n):
        print("{} : {}".format(i, j))

        diagram_a = diagrams["emm_moon_" + str(i) + " Contact Analysis.csv"]
        diagram_b = diagrams["emm_mars_" + str(j) + " Contact Analysis.csv"]
        
        diagram_a_clique = diagrams_clique["emm_moon_" + str(i) + " Contact Analysis.csv"]
        diagram_b_clique = diagrams_clique["emm_mars_" + str(j) + " Contact Analysis.csv"]

        wdist = d.wasserstein_distance(diagram_a[dim], diagram_b[dim], q=2)
        print("2-Wasserstein distance between {}-dimensional persistence diagrams: {}".format(dim, wdist))

        bdist = d.bottleneck_distance(diagram_a[dim], diagram_b[dim])
        print("Bottleneck distance between {}-dimensional persistence diagrams: {}".format(dim, bdist))
        
        wdist_clique = d.wasserstein_distance(diagram_a_clique[dim], diagram_b_clique[dim], q=2)
        print("2-Wasserstein distance between {}-dimensional persistence diagrams (Clique): {}".format(dim, wdist_clique))

        bdist_clique = d.bottleneck_distance(diagram_a_clique[dim], diagram_b_clique[dim])
        print("Bottleneck distance between {}-dimensional persistence diagrams (Clique): {}".format(dim, bdist_clique))
        
        